In [1]:
import urllib.request, json , time, os, difflib, itertools
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime

In [2]:
try:
    import httplib
except:
    import http.client as httplib

def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        # print("True")
        return True
    except:
        conn.close()
        # print("False")
        return False

In [3]:
check_internet()

True

In [4]:
def get_historic_price(query_url,json_path,csv_path):
    
    stock_id=query_url.split("&period")[0].split("symbol=")[1]
    
    #стираем файлы если уже есть
    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        os.remove(csv_path+stock_id+'.csv')
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)

    try:
        with urllib.request.urlopen(query_url) as url:
            parsed = json.loads(url.read().decode())
    
    except:
        print("|||  Historical data of "+stock_id+" doesn't exist")
        return
    
    else:
        if os.path.exists(json_path+stock_id+'.json') and os.stat(json_path+stock_id+'.json').st_size != 0:
            os.remove(json_path+stock_id+'.json')
        
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)
        
        try:
            Date=[]
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%Y-%m-%d'))

            #Low=parsed['chart']['result'][0]['indicators']['quote'][0]['low']
            #Open=parsed['chart']['result'][0]['indicators']['quote'][0]['open']
            #Volume=parsed['chart']['result'][0]['indicators']['quote'][0]['volume']
            #High=parsed['chart']['result'][0]['indicators']['quote'][0]['high']
            Close=parsed['chart']['result'][0]['indicators']['quote'][0]['close']
            #Adjusted_Close=parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

            df=pd.DataFrame(list(zip(Date,Close)),columns =['Date',stock_id[1:]])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
        
        except:
            print(">>>  Historical data of "+stock_id+" could not be saved")
        
        return

In [5]:
json_path = "data/json/"
csv_path = "data/csv/"

In [6]:
if not os.path.isdir(json_path):
    os.makedirs(json_path)
if not os.path.isdir(csv_path):
    os.makedirs(csv_path)

In [7]:
from tickers import tickers_dict

In [8]:
# Период с 2016-06-30 по 2021-06-30, интервал 1 неделя (допустмые значения [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo])
timestamp_start = str(int(datetime.strptime('2018-06-30', "%Y-%m-%d").timestamp()))
timestamp_end = str(int(datetime.strptime('2019-06-30', "%Y-%m-%d").timestamp()))
interval = '1d'

In [9]:
timestamp_start

'1530306000'

In [23]:
query_urls=[]

for ticker in tickers_dict.values():
    query_urls.append("https://query1.finance.yahoo.com/v8/finance/chart/"+ticker[0]+"?symbol="+ticker[0]+"&period1="+timestamp_start+"&period2="+timestamp_end+"&interval="+interval)

In [24]:
query_urls

['https://query1.finance.yahoo.com/v8/finance/chart/^IXIC?symbol=^IXIC&period1=1530306000&period2=1561842000&interval=1d',
 'https://query1.finance.yahoo.com/v8/finance/chart/^NDX?symbol=^NDX&period1=1530306000&period2=1561842000&interval=1d',
 'https://query1.finance.yahoo.com/v8/finance/chart/^BANK?symbol=^BANK&period1=1530306000&period2=1561842000&interval=1d',
 'https://query1.finance.yahoo.com/v8/finance/chart/^NBI?symbol=^NBI&period1=1530306000&period2=1561842000&interval=1d',
 'https://query1.finance.yahoo.com/v8/finance/chart/^IXCO?symbol=^IXCO&period1=1530306000&period2=1561842000&interval=1d',
 'https://query1.finance.yahoo.com/v8/finance/chart/^IXHC?symbol=^IXHC&period1=1530306000&period2=1561842000&interval=1d',
 'https://query1.finance.yahoo.com/v8/finance/chart/^INDS?symbol=^INDS&period1=1530306000&period2=1561842000&interval=1d',
 'https://query1.finance.yahoo.com/v8/finance/chart/^INSR?symbol=^INSR&period1=1530306000&period2=1561842000&interval=1d',
 'https://query1.fin

In [25]:
%%time
with Pool(processes=10) as pool:
    pool.starmap(get_historic_price, zip(query_urls, itertools.repeat(json_path), itertools.repeat(csv_path)))

>>>  Historical data of ^OFIN saved
>>>  Historical data of ^NBI saved
>>>  Historical data of ^IXTC saved
>>>  Historical data of ^INDS saved
>>>  Historical data of ^IXHC saved
>>>  Historical data of ^INSR saved
>>>  Historical data of ^IXCO saved
>>>  Historical data of ^IXIC saved
>>>  Historical data of ^NDX saved
>>>  Historical data of ^BANK saved
>>>  Historical data of ^TRAN saved
>>>  Historical data of ^DJI saved
>>>  Historical data of ^NYA saved
CPU times: user 374 ms, sys: 7.23 ms, total: 381 ms
Wall time: 701 ms
